# LattesXML ⛏  🧐
## Notebook para extração de dados de arquivos XML do currículo Lattes/CNPq
#### Autor: Gabriel Torell | torelli@yahoo.com
#### Ajude a desenvolver essa ferramenta: http://github.com/gtorelli

Este programa é um software livre; você pode redistribuí-lo e/ou
modificá-lo sob os termos da Licença Pública Geral GNU como publicada
pela Free Software Foundation; na versão 3 da Licença, ou
(a seu critério) qualquer versão posterior.

Este programa é distribuído na esperança de que possa ser útil,
mas SEM NENHUMA GARANTIA; sem uma garantia implícita de ADEQUAÇÃO
a qualquer MERCADO ou APLICAÇÃO EM PARTICULAR. Veja a
Licença Pública Geral GNU para mais detalhes.

Você deve ter recebido uma cópia da Licença Pública Geral GNU junto
com este programa. Se não, veja <http://www.gnu.org/licenses/>.

### Como utilizar?
1. Faça o download dos arquivos XML utilizando a plataforma Lattes;
2. Coloque os arquivos dentro de um diretório chamado "XML";
3. Crie um diretório chamado "relatorios";
3. Execute o script.

Os resultados são gerados em formato html e colocados no diretório "relatorios".


In [1]:
# Importa Bibliotecas
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
from IPython.core.display import display, HTML
import glob
import os
import re

In [2]:
# Configura colunas para tamanho máximo
pd.set_option('display.max_colwidth', None)
# Exibe Todas as Linhas do DataFrame
pd.set_option('display.max_rows', None)

In [3]:
def ler_lattes(arquivo):
    content = []
    with open("XML/"+arquivo, "r", encoding="ISO-8859-1") as file:
        content = file.readlines()
        content = "".join(content)
        return bs(content, "lxml")

In [4]:
luciano = ler_lattes('luciano_curriculo.xml')
paulo = ler_lattes('paulo_curriculo.xml')
marco = ler_lattes('marco_curriculo.xml')
moises = ler_lattes('moises_curriculo.xml')
rigoberto = ler_lattes('rigoberto_curriculo.xml')

## ARTIGOS EM CONGRESSOS

In [5]:
def limpeza(n):
    return re.findall('nome-completo-do-autor\s*=\s*\"(.+?)\"', str(n))

In [6]:
def autores_artigos(membro):
    lista = []
    df_tmp = pd.DataFrame()
    
    for link in membro.find_all("artigo-publicado"):
        lista.append(link.find_all("autores"))
    autores = list(map(limpeza, lista))
    df_tmp['AUTORES'] = pd.Series(autores)
    df_tmp['AUTORES'] = df_tmp['AUTORES']
    df_tmp['AUTORES'] = [', '.join(map(str,l)) for l in df_tmp['AUTORES']]
    df_tmp['AUTORES'] = df_tmp['AUTORES'].str.title()
    return df_tmp

In [7]:
def titulo_artigos(membro):
    lista = []
    df_tmp = pd.DataFrame()

    for titulo in membro.find_all("dados-basicos-do-artigo"):
        lista.append(titulo.get("titulo-do-artigo"))
    df_tmp['TITULO'] = pd.Series(lista).str.upper() 
    return df_tmp

In [8]:
def periodico_artigos(membro):
    lista = []
    df_tmp = pd.DataFrame()
    
    for titulo in membro.find_all("detalhamento-do-artigo"):
        lista.append(titulo.get("titulo-do-periodico-ou-revista"))
    df_tmp['PERIODICO'] = pd.Series(lista).str.title()
    return df_tmp

In [9]:
def doi_artigos(membro):
    lista = []
    df_tmp = pd.DataFrame()
    
    for link in membro.find_all("dados-basicos-do-artigo"):
        lista.append("<a href=http://dx.doi.org/{}".format(link.get("doi")+">"+link.get("doi")+"</a>"))
    df_tmp['DOI'] = pd.Series(lista)
    return df_tmp

In [10]:
def ano_artigos(membro):
    lista = []
    df_tmp = pd.DataFrame()
    for link in membro.find_all("dados-basicos-do-artigo"):
        lista.append(link.get("ano-do-artigo"))
    df_tmp['ANO'] = pd.Series(lista)
    return df_tmp

In [11]:
def processar_dataframe(df):
    df_tmp = pd.DataFrame()
    df_tmp = df[df['DOI'] != '<a href=http://dx.doi.org/></a>']
    df_tmp = df_tmp[df_tmp['ANO'] >= '2010']
    df_tmp = df_tmp.drop_duplicates(subset=['DOI'])
    df_tmp = df_tmp.sort_values(by=['ANO']).reset_index(drop=True)
    return df_tmp

In [12]:
# RELATORIO ARTIGOS PROFESSOR MARCO
marco_df = pd.concat([autores_artigos(marco), titulo_artigos(marco), doi_artigos(marco), ano_artigos(marco)], axis=1, join="inner")
processar_dataframe(marco_df).to_html('relatorios/marco.html', escape=False)

In [13]:
# RELATORIO ARTIGOS PROFESSOR RIGOBERTO
rigoberto_df = pd.concat([autores_artigos(rigoberto), titulo_artigos(rigoberto), doi_artigos(rigoberto), ano_artigos(rigoberto)], axis=1)
processar_dataframe(rigoberto_df).to_html('relatorios/rigoberto.html', escape=False)

In [14]:
# RELATORIO ARTIGOS PROFESSOR LUCIANO
luciano_df = pd.concat([autores_artigos(luciano), titulo_artigos(luciano), doi_artigos(luciano), ano_artigos(luciano)], axis=1)
processar_dataframe(luciano_df).to_html('relatorios/luciano.html', escape=False)

In [15]:
# RELATORIO ARTIGOS PROFESSOR MOISES
moises_df = pd.concat([autores_artigos(moises), titulo_artigos(moises), doi_artigos(moises), ano_artigos(moises)], axis=1)
processar_dataframe(moises_df).to_html('relatorios/moises.html', escape=False)

In [16]:
# RELATORIO ARTIGOS PROFESSOR PAULO
paulo_df = pd.concat([autores_artigos(paulo), titulo_artigos(paulo), doi_artigos(paulo), ano_artigos(paulo)], axis=1)
processar_dataframe(paulo_df).to_html('relatorios/paulo.html', escape=False)

In [17]:
# DATAFRAME COM TODAS AS PUBLICAÇÕES
final_df = pd.concat([marco_df, rigoberto_df, luciano_df, moises_df, paulo_df])
final_df = processar_dataframe(final_df)
final_df.to_html('relatorios/artigos_final.html', escape=False)
final_df

,AUTORES,TITULO,DOI,ANO
0,"Marco Jose Da Silva, Sebastian Thiele, Lokman Abdulkareem, Barry Azzopardi, Uwe Hampel",HIGH-RESOLUTION GAS-OIL TWO-PHASE FLOW VISUALIZATION WITH A CAPACITANCE WIRE-MESH SENSOR,<a href=http://dx.doi.org/10.1016/j.flowmeasinst.2009.12.003>10.1016/j.flowmeasinst.2009.12.003</a>,2010
1,"Moisés Alves Marcelino Neto, Barbosa Jr., Jader R.","SOLUBILITY, DENSITY AND VISCOSITY OF MIXTURES OF ISOBUTANE (R-600A) AND A LINEAR ALKYLBENZENE LUBRICANT OIL",<a href=http://dx.doi.org/10.1016/j.fluid.2009.12.029>10.1016/j.fluid.2009.12.029</a>,2010
2,"Moisés Alves Marcelino Neto, Jader R. Barbosa Jr",VISCOSITY BEHAVIOR OF MIXTURES OF CO2 AND LUBRICANT OIL,<a href=http://dx.doi.org/10.1590/s1678-58782010000400007>10.1590/s1678-58782010000400007</a>,2010
3,"Moisés Alves Marcelino Neto, Jader R. Barbosa Jr",MODELING OF STATE AND THERMODYNAMIC CYCLE PROPERTIES OF HFO-1234YF USING A CUBIC EQUATION OF STATE,<a href=http://dx.doi.org/10.1590/S1678-58782010000500005>10.1590/S1678-58782010000500005</a>,2010
4,"Paulo Henrique Dias Dos Santos, Bazzo, Edson, Becker, Susanne, Kulenovic, Rudi, Mertz, Rainer",DEVELOPMENT OF LHPS WITH CERAMIC WICK,<a href=http://dx.doi.org/10.1016/j.applthermaleng.2010.04.010>10.1016/j.applthermaleng.2010.04.010</a>,2010
5,"Henrique Stel De Azevedo, Rigoberto Eleazar Melgarejo Morales, Admilson T. Franco, Silvio Junqueira",NUMERICAL AND EXPERIMENTAL ANALYSIS OF TURBULENT FLOW IN CORRUGATED PIPES,<a href=http://dx.doi.org/10.1115/1.4002035>10.1115/1.4002035</a>,2010
6,"Andre Bieberle, Markus Schubert, Marco Jose Da Silva, Uwe Hampel",MEASUREMENT OF LIQUID DISTRIBUTIONS IN PARTICLE PACKINGS USING WIRE-MESH SENSOR VERSUS TRANSMISSION TOMOGRAPHIC IMAGING,<a href=http://dx.doi.org/10.1021/ie100900g>10.1021/ie100900g</a>,2010
7,"Valente Hernandez Perez, Barry Azzopardi, Ryuhei Kaji, Marco Jose Da Silva, Matthias Beyer, Uwe Hampel",WISP-LIKE STRUCTURES IN VERTICAL GAS-LIQUID PIPE FLOW REVEALED BY WIRE MESH SENSOR STUDIES,<a href=http://dx.doi.org/10.1016/j.ijmultiphaseflow.2010.08.002>10.1016/j.ijmultiphaseflow.2010.08.002</a>,2010
8,"Marco Jose Da Silva, Sebastian Thiele, Thomas Hoehne, Roman Vaibar, Uwe Hampel",EXPERIMENTAL STUDIES AND CFD CALCULATIONS FOR BUOYANCY DRIVEN MIXING PHENOMENA,<a href=http://dx.doi.org/10.1016/j.nucengdes.2009.11.023>10.1016/j.nucengdes.2009.11.023</a>,2010
9,"Barry Azzopardi, Lokman Abdulkareem, Donglin Zhao, Sebastian Thiele, Marco Jose Da Silva, Matthias Beyer, Andrew Hunt",COMPARISON BETWEEN ELECTRICAL CAPACITANCE TOMOGRAPHY AND WIRE MESH SENSOR OUTPUT FOR AIR/SILICONE OIL FLOW IN A VERTICAL PIPE,<a href=http://dx.doi.org/10.1021/ie901949z>10.1021/ie901949z</a>,2010
